In [25]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fmin
import warnings
warnings.filterwarnings("ignore")

The following code is to train and test the a strategy for investment :

	* The objective is to use quant index to adjust the weight of our strategy, and find the optimal parameters for the strategy. 
    
	* Train a model based on the 2011 - 2018 data and test them on 2019 - 2021 data，data is from EIA with expiry date each months, roll F_1 to F_2 at the closing price on the 5th business day prior to the futures expiration.
    
	* Trades are entered at the closing price on the day after the signal is calculated, signal can determine the position tomorrow. (D1 data - D2 start implement - D2 end profit)
    
	* Transaction cost is $0.01/bbl for both buy and sell contract.

In [26]:
# read future price data
WTI_df = pd.read_excel (r'/Users/shengbo/Desktop/PET_PRI_FUT_S1_D.xls', 
                        sheet_name='Data 1',index_col=0,header=2,names=['F1','F2','F3','F4'])
# the date for future expiration
expir_df = pd.read_excel (r'/Users/shengbo/Desktop/Expiry Calendar.xlsx', header=0)

# apply our strategy in this period
start_date = '2019-1-1'
end_date = '2021-12-31'
WTI = WTI_df.loc[(WTI_df.index > start_date) & (WTI_df.index <= end_date)]
wti_exp = expir_df.WTI[(expir_df.WTI > start_date) & (expir_df.WTI <= end_date)]
day = len(WTI)

In [27]:
# add column to classify expiration date, and then trading date
WTI['expiration']=np.zeros(len(WTI))
for i in WTI.index:
    if i in np.array(wti_exp):
        WTI.loc[i,'expiration']=1

WTI['trade']=np.concatenate([np.array(WTI['expiration'])[5:],np.zeros(5)]) # trade date is based on expiration date

In [28]:
WTI.head(15)
# F1-F4 stand for the price of contracts expire in the following four months. 

,F1,F2,F3,F4,expiration,trade
2019-01-02,46.54,46.86,47.24,47.70,0.0,0.0
2019-01-03,47.09,47.40,47.77,48.22,0.0,0.0
2019-01-04,47.96,48.28,48.65,49.10,0.0,0.0
2019-01-07,48.52,48.82,49.17,49.60,0.0,0.0
2019-01-08,49.78,50.11,50.49,50.93,0.0,0.0
2019-01-09,52.36,52.69,53.07,53.51,0.0,0.0
2019-01-10,52.59,52.91,53.28,53.73,0.0,0.0
2019-01-11,51.59,51.91,52.27,52.70,0.0,0.0
2019-01-14,50.51,50.80,51.14,51.55,0.0,1.0
2019-01-15,52.11,52.39,52.70,53.08,0.0,0.0


In [29]:
# variable for Carry strategy
WTI['F1F4_diff'] = WTI['F1']-WTI['F4']

# define a function for carry strategy, return a column with position, '1' for hold and '0' otherwise.
# Return a term for final position, Inhand=1 stand for long and -1 for short.
# Calculation is based on the future price at the current day, suppose strategy will actually implement the next day
def Carry_strat(epi):
    Inhand = 0
    Inhand_td=0
    position = np.zeros(day)
    transaction = np.zeros(day)
    for i in range(day):
        position[i] = Inhand 
        if WTI['F1F4_diff'].iloc[i]>epi: #buy, calculate when this day(denote D1) data is available 
            Inhand_td = 1 # implement this strategy at D2, D2 inhand will be 1
        elif WTI['F1F4_diff'].iloc[i]<-epi: #sell
            Inhand_td = -1
        else:
            Inhand_td = 0
            
        if Inhand_td != Inhand:
            transaction[i] = 0.01 * abs(Inhand_td - Inhand) # transaction cost depends on numbers of transaction
            
        Inhand = Inhand_td # update the position
    return position, transaction 

In [30]:
# For Momentum strategy, need former data for the moving average
def Moving_Ave(n, col='F1'):
    Start_index = WTI_df.index.get_loc(WTI.index[0])
    Extend_index = Start_index-n+1
    # Add n-1 days of F1 data to finish moving average
    End_index = WTI_df.index.get_loc(WTI.index[-1])+1
    WTI_extended = WTI_df.iloc[Extend_index:End_index]
    MA = WTI_extended[col].rolling(n).mean()[n-1:] 
    return MA

# Compare F1 with MovingAverage to detrmine our position.
def Momentum_strat(n):
    Inhand = 0
    Inhand_td=0
    MA = Moving_Ave(n)
    position = np.zeros(day)
    transaction = np.zeros(day)
    for i in range(day):
        position[i] = Inhand
        if WTI['F1'].iloc[i] > MA.iloc[i]: # long
            Inhand_td = 1
        elif WTI['F1'].iloc[i] < MA.iloc[i]: # short
            Inhand_td = -1
            
        if Inhand_td != Inhand:
            transaction[i] = 0.01 * abs(Inhand_td - Inhand)
        Inhand = Inhand_td
    return position, transaction

In [31]:
def EWMA(n, col='F1'):
    Start_index = WTI_df.index.get_loc(WTI.index[0])
    Extend_index = Start_index-n+1
    # Add n-1 days of F1 data to finish moving average
    End_index = WTI_df.index.get_loc(WTI.index[-1])+1
    
    WTI_extended = WTI_df.iloc[Extend_index:End_index]
    
    EWMA = ewma(WTI_extended[col], span=n).mean()[n-1:] 
    return EWMA

In [32]:
def daily_pl(para, strat):
    pos,trans = strat(para)
    day = len(WTI)
    stock_price = np.zeros(day)
    daily_pl = np.zeros(day)
    holding_F1 = True
    for i in range(day):
        # determine the stock we hold each day
        if WTI['expiration'].iloc[i] == 1 or WTI['trade'].iloc[i] == 1:
            stock_price[i] =  WTI['F2'].iloc[i]
            holding_F1 = not holding_F1
        else:
            stock_price[i] = WTI['F2'].iloc[i] if holding_F1 == False else WTI['F1'].iloc[i]
            
            
        if i > 0:
            if pos[i] == 1:
                if WTI['trade'].iloc[i] == 1:
                    daily_pl[i] = WTI['F1'].iloc[i]-WTI['F1'].iloc[i-1]
                    # profit in trade day is actually the price change in F1
                else:
                    # otherwise, just change pf price in our holding stock 
                    daily_pl[i] = stock_price[i]-stock_price[i-1]
            elif pos[i] == -1:
                if WTI['trade'].iloc[i] == 1:
                    daily_pl[i] = WTI['F1'].iloc[i-1]-WTI['F1'].iloc[i]
                else:
                    daily_pl[i] = stock_price[i-1]-stock_price[i]
    daily_pl- trans    
        
    return daily_pl


In [33]:
# input the parameter for strategy and return the sharpe ratio during the period
def Sharpe(para,strat):
    daily_pl_temp = daily_pl(para,strat)
    stdev_daly_pl = np.std(daily_pl_temp)
    cum_pl = daily_pl_temp.cumsum()
    AveAnu_pl = cum_pl[-1]/12
    SharpeR = AveAnu_pl / ( stdev_daly_pl * np.sqrt(250) ) # 250 business day per year

    return SharpeR 

In [34]:
def Mom_carry(n):
    Inhand = 0
    C14 = Moving_Ave(n,'F1')-Moving_Ave(n,'F4')
    position = np.zeros(day)
    transaction = np.zeros(day)
    for i in range(day):
        position[i] = Inhand
        if WTI['F1'].iloc[i] - WTI['F4'].iloc[i] > C14.iloc[i]: # long
            Inhand_td = 1
        elif WTI['F1'].iloc[i] - WTI['F4'].iloc[i] < C14.iloc[i]: # short
            Inhand_td = -1
            
        if Inhand_td != Inhand:
            transaction[i] = 0.01 * abs(Inhand_td - Inhand)
        Inhand = Inhand_td
    return position, transaction

In [35]:
def EWMA_carry(n):
    Inhand = 0
    C14 = EWMA(n,'F1')-EWMA(n,'F4')
    position = np.zeros(day)
    transaction = np.zeros(day)
    for i in range(day):
        position[i] = Inhand
        if WTI['F1'].iloc[i] - WTI['F4'].iloc[i] > C14.iloc[i]: # long
            Inhand_td = 1
        elif WTI['F1'].iloc[i] - WTI['F4'].iloc[i] < C14.iloc[i]: # short
            Inhand_td = -1
            
        if Inhand_td != Inhand:
            transaction[i] = 0.01 * abs(Inhand_td - Inhand)
        Inhand = Inhand_td
    return position, transaction

In [58]:
nEC_opt = max(np.arange(5,41,1),key = lambda k: Sharpe(k,EWMA_carry))
print('The optimal n for Carry-Momentum strategy is',nEC_opt)

ECopt_daily_PL=daily_pl(nEC_opt, EWMA_carry)
EL_EC=ECopt_daily_PL.cumsum()

print ('The annualized return of the combined strategy is $', EL_EC[-1]/12,'per barrel')
print ('The Sharpe ratio of the combined strategy is ', Sharpe(nEC_opt,EWMA_carry))


The optimal n for Carry-Momentum strategy is 14
The annualized return of the combined strategy is $ 8.537499999999998 per barrel
The Sharpe ratio of the combined strategy is  0.3742960988602206
